## Introduction to Data Science

#### University of Redlands - DATA 101
#### Prof: Joanna Bieri [joanna_bieri@redlands.edu](mailto:joanna_bieri@redlands.edu)
#### [Class Website: data101.joannabieri.com](https://joannabieri.com/data101.html)

---------------------------------------
# Homework Day 16
---------------------------------------

GOALS:

1. Work to understand when a linear model vs. non-linear model is a good choice.
2. Practice plotting residuals and regression lines.
3. Do an analysis using non-linear regression.


----------------------------------------------------------

This homework has **2 Questions** and **3 Exercises**

## Important Information

- Email: [joanna_bieri@redlands.edu](mailto:joanna_bieri@redlands.edu)
- Office Hours: Duke 209 <a href="https://joannabieri.com/schedule.html"> Click Here for Joanna's Schedule</a>


## Announcements

**Come to Lab!** If you need help we are here to help!

## Day 16 Assignment - same drill.

1. Make sure you can **Fork** and **Clone** the Day16 repo from [Redlands-DATA101](https://github.com/Redlands-DATA101)
2. Open the file Day16-HW.ipynb and start doing the problems.
    * You can do these problems as you follow along with the lecture notes and video.
3. Get as far as you can before class.
4. Submit what you have so far **Commit** and **Push** to Git.
5. Take the daily check in quiz on **Canvas**.
7. Come to class with lots of questions!

In [28]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.defaule = 'colab'

from itables import show

# This stops a few warning messages from showing
pd.options.mode.chained_assignment = None 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Machine Learning Packages
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression 

## Paris Paintings Data - Load the data

To explore the ideas of modeling data we will use the Paris Paintings dataset.

- Source: Printed catalogs of 28 auction sales in Paris, 1764 - 1780 (Historical Data)
- Data curators Sandra van Ginhoven and Hilary Coe Cronheim (who were PhD students in the Duke Art, Law, and Markets Initiative at the time of putting together this dataset) translated and tabulated the catalogs
-  3393 paintings, their prices, and descriptive details from sales catalogs over 60 variables

[Variables in Paris Paintings Data](https://www2.stat.duke.edu/~cr173/Sta112_Fa16/data/paris_paintings.html)

This lab follows the Data Science in a Box units "Unit 4 - Deck 3: Modeling nonlinear relationships " by Mine Çetinkaya-Rundel. It has been updated for our class and translated to Python by Joanna Bieri.

In [29]:
file_location = 'https://joannabieri.com/introdatascience/data/paris-paintings.csv'
DF_raw_paintings = pd.read_csv(file_location,na_filter=False)

In [30]:
show(DF_raw_paintings)

In [31]:
# Make a copy of the data that we can start working on
DF = DF_raw_paintings.copy()

# Do something about all those different NaNs
DF.replace('',np.nan,inplace=True)
DF.replace('n/a',np.nan,inplace=True)
DF.replace('NaN',np.nan,inplace=True)

## Explore Linearity

Fit a model for Price as a function of size. Let's redo that analysis except this time focus only on paintings with area of less than 10,000 inches squared.

In [32]:
#| code-fold: true
#| code-summary: "Answer to Exercise 2 Day 15 for paintings < 10000 in^2"

# Get the columns I care about
my_columns = ['Surface','price']
DF_model2 = DF[my_columns]

# Do some preprocessing - drop NA and make the Surface variable a float
DF_model2.dropna(inplace=True)
DF_model2['Surface'] = DF_model2['Surface'].apply(lambda x: float(x))

# Mask the data for surface area less than 10,000
mask = DF_model2['Surface'] <= 10_000
DF_model2 = DF_model2[mask]

# Make a Scatter plot
fig = px.scatter(DF_model2,
                 x='Surface',
                 y='price',
                 color_discrete_sequence=['black'],
                 opacity=0.2)

fig.update_layout(template="ggplot2",
                  title='Price as a function of Surface Area - total area < 5000 in sq',
                  title_x=0.5,
                  xaxis_title='Surface Area',
                  yaxis_title='Price')


fig.show()

# Create the X and y variables for Linear Regression
X = DF_model2['Surface'].values.reshape(-1,1)
y = DF_model2['price'].values

# Create linear regression object - a random straight line
LM = LinearRegression()
# Train the model using the data
LM.fit(X, y)

# The score from my model
print('Model Score:')
print(LM.score(X,y))

Model Score:
0.014875957473786783


**Exercise 1** Create a Residual plot for the model above.

1. Get the predictions - store these in a column in the data frame

```{python}
LM.predict(X)
```

2. Calculate the residuals - store these in a column in the data frame
```{python}
'Residual' = 'Real Value in the Data' - 'Value Predicted by LM'
```

3. Plot the result
```{python}
px.scatter(df,x='Value Predicted by LM',y='Residual')
```


See if you can recreate the plot shown in the lecture.

In [33]:
DF_model2['Predicted_Price'] = LM.predict(X)
DF_model2['Residual'] = DF_model2['price']-DF_model2['Predicted_Price']

In [34]:
# Create a residual plot
fig = px.scatter(
    DF_model2,
    x='Predicted_Price',
    y='Residual',
    title="Residual as a function of predicted price",
    template="ggplot2"
)


fig.update_layout(
    xaxis_title="Predicted Price",
    yaxis_title="Residuals",
    yaxis=dict(zeroline=True, zerolinewidth=1.5, zerolinecolor="black")
)

fig.show()


**Q1** What do you see here? Does this residual data seem uniformly distributed?


The resiudals are not unfiformly distributed. This indicates that the model is not capturing some non-linearity in the data. We can also see that residuals increase for lower predicted prices as well as the spread of residuals. Overall, the plot indicates an alternative method that is not linear regression should be investigated.

## From Lecture - Consider the skew in the data

In [35]:
# Plot a histogram - notice that it is very skewed
fig = px.histogram(DF_model2,x='price',nbins =30)
fig.show()

In [36]:
# Us numpy to take the natural log of the data - removing the exponential decay
DF_model2['log_price'] = np.log(DF_model2['price'])

In [37]:
# Plot a histogram of the log(price) - notice we have removed the skew
fig = px.histogram(DF_model2,x='log_price',nbins =30)

fig.show()

In [38]:
# Look at the scatter plot of the log(price)
fig = px.scatter(DF_model2,
                 x='Surface',
                 y='log_price',
                 color_discrete_sequence=['black'],
                 opacity=0.2)

fig.update_layout(template="ggplot2",
                  title='log(Price) as a function of Surface Area - total area < 5000 in sq',
                  title_x=0.5,
                  xaxis_title='Surface Area',
                  yaxis_title='log(Price)')


fig.show()

**Q2** What is different about the histogram and the scatter plots after taking the natural log?

The histogram becomes much more symmetrical and represents more of a normal distribution. Whilst in the scatter plot the spread of points is much more uniform and the relationship appears to be more linear. lol

**Exercise 2:** Redo the linear regression analysis except this time use the log_price.

- Find the linear regression model (LM)
- Calculate the residual
- Plot the Residual as a function of the predicted log price
- Plot a scatter plot of the data with the linear regression line added.

*HINT* You can see my results in the lecture notes!

In [40]:
X_log = DF_model2['Surface'].values.reshape(-1, 1)
y_log = DF_model2['log_price'].values

LM_log = LinearRegression()

LM_log.fit(X_log, y_log)

print('Model Score (Log Price):')
print(LM_log.score(X_log, y_log))

DF_model2['Predicted_log_price'] = LM_log.predict(X_log)

DF_model2['Residual_log'] = DF_model2['log_price'] - DF_model2['Predicted_log_price']

fig_residual = px.scatter(DF_model2,
                          x='Predicted_log_price',
                          y='Residual_log',
                          color_discrete_sequence=['red'],
                          opacity=0.6)

fig_residual.update_layout(template="ggplot2",
                           title='Residuals as a function of Predicted Log Price',
                           title_x=0.5,
                           xaxis_title='Predicted Log Price',
                           yaxis_title='Residuals')
fig_residual.show()

fig_regression = px.scatter(DF_model2,
                            x='Surface',
                            y='log_price',
                            color_discrete_sequence=['blue'],
                            opacity=0.5)


fig_regression.add_scatter(x=x_range, y=y_range, mode='lines', name='Regression Line', line=dict(color='black'))

fig_regression.update_layout(template="ggplot2",
                              title='Log(Price) as a function of Surface Area with Regression Line',
                              title_x=0.5,
                              xaxis_title='Surface Area',
                              yaxis_title='Log(Price)')
fig_regression.show()

Model Score (Log Price):
0.013268243020669424


### What did we learn...

What is the model telling me?

$$ \hat{\log(price)} = 4.912 + 0.00024(Surface Area)$$

so we can calculate (see the lecture for details!)

$$ (SA+1) \sim 1.0002400288023041 * SA$$

This tells us that increase the area of the painting by one square inch increases the price by a factor of 1.0002400288023041 or about 0.024%.

There is a small positive increase in the price as the surface area increases, on average. 

Can we predict the price using the surface area? Look at LM.score(X,y)...

It does not appear that our logistic regression is a good predictor of the price. Even though it looks like we captured a good linear relationship, we do not have a good predictor. The scatter is still very large!

BUT - we are still able to see a linear trend in the model. There is a relationship here even though the data is very noisy!

**Exercise 3** Redo the full analysis except this time try using just height to predict price.

- Do a standard linear regression of the height vs. the price (without log) discuss the results. This should include a plot of the residuals and a plot showing the linear fit. You should also talk about what the score, intercept and coefficient of the model are telling you. EG. As the height increases by 1in the price.....
- Do a linear regression of the height vs. log_price and discuss the results. This should include a plot of the residuals and a plot showing the linear fit. You should also talk about what the score, intercept and coefficient of the model are telling you. EG. As the height increases by 1in the price..... Remember in this case you have to use the rules of logs and exponents to interpret the results.

Which of these models do you think is doing a better job of capturing the functional relationships in the price vs height data? Why?

In [41]:
DF_model2

,Surface,price,Predicted_Price,Residual,log_price,Predicted_log_price,Residual_log
0,1091.5,360.0,904.234443,-544.234443,5.886104,5.170882,0.715222
1,252.0,6.0,683.985984,-677.985984,1.791759,4.971419,-3.179660
2,208.0,12.0,672.442289,-660.442289,2.484907,4.960965,-2.476058
3,252.0,6.0,683.985984,-677.985984,1.791759,4.971419,-3.179660
4,252.0,6.0,683.985984,-677.985984,1.791759,4.971419,-3.179660
...,...,...,...,...,...,...,...
3388,387.0,18.0,719.404140,-701.404140,2.890372,5.003495,-2.113123
3389,214.5,25.0,674.147607,-649.147607,3.218876,4.962509,-1.743634
3390,720.0,80.0,806.768925,-726.768925,4.382027,5.082615,-0.700588
3391,621.0,5.0,780.795610,-775.795610,1.609438,5.059093,-3.449655
